In [52]:
# DO NOT modify this cell. 
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [1]:
# test case 1 vertical
filename = "CAS.pgm"
verticalSeam2Remove = 20
horizontalSeam2Remove = 0

In [3]:
# test case 2 vertical and horizontal
filename = "CAS_2.pgm"
verticalSeam2Remove = 100
horizontalSeam2Remove = 50

In [2]:
# function to read the input image
def parse_pgm(input_filename):
    with open(input_filename, 'r') as file:
        # Read and discard the header and comment lines
        magic = file.readline()
        comment = file.readline()

        # Read width and height
        width, height = map(int, file.readline().split())

        # Read and discard the max value indicator
        max_color_value = file.readline()

        # Read pixel values into a flat list
        pixel_values = list(map(int, file.read().split()))

    # Reshape the flat list into a 2D array
    image = [pixel_values[i * width:(i + 1) * width] for i in range(height)]

    return image


# function to create an output file
def output_file(image_array, out_file_name):
    with open(out_file_name, 'w') as file:
        # header
        file.write('P2\n')
        file.write(f'{len(image_array[0])} {len(image_array)}\n')
        file.write('255\n')

        # pixel body
        for row in image_array:
            for column in row:
                file.write(str(column))
                file.write(" ")
            file.write("\n")


def calcEnergyMap(image):
    # create an empty map the same size as the image array 
    height = len(image)
    width = len(image[0])
    energyMap = [[0] * width for _ in range(height)]
    
    # iterate thorugh the image array
    for y in range(height):
        for x in range(width):

            # Calculate energy in x direction
            # left edge
            if (x == 0):
                dx = abs(image[y][x] - image[y][x+1])
            elif (x == width - 1):
                # right edge
                dx = abs(image[y][x] - image[y][x-1])
            else:
                # everything else
                dx = abs(image[y][x] - image[y][x-1]) + abs(image[y][x] - image[y][x+1])
        
            # Calculate energy in y direction
            # top edge
            if (y == 0):
                dy = abs(image[y][x] - image[y + 1][x])
            # bottom edge
            elif (y == height - 1):
                dy = abs(image[y][x] - image[y - 1][x])
            # everything else
            else:
                dy = abs(image[y][x] - image[y - 1][x]) + abs(image[y][x] - image[y + 1][x])

            energyMap[y][x] = dy + dx

    return energyMap


def locate_seam(image_array):
    # Calculate the energy map
    energy_map = calcEnergyMap(image_array)
    height, width = len(energy_map), len(energy_map[0])

    # create an empty array the same size as the energy map
    total = [[0 for _ in range(width)] for _ in range(height)]

    # Copy the first row since there's no row above it
    total[0] = energy_map[0]

    # Calculate cumulative energy values for each pixel, exculding the first and last
    for y in range(1, height):
        for x in range(width):
            # Calculate cumulative energy value for the current pixel
            if x == 0:  # Left edge
                total[y][x] = energy_map[y][x] + min(total[y-1][x], total[y-1][x+1])
            elif x == width - 1:  # Right edge
                total[y][x] = energy_map[y][x] + min(total[y-1][x-1], total[y-1][x])
            else:  # Everything else
                total[y][x] = energy_map[y][x] + min(total[y-1][x-1], total[y-1][x], total[y-1][x+1])

    # Find the index of the minimum energy value in the last row
    min_energy_index = total[height-1].index(min(total[height-1]))

    # Backtrack to find the minimum seam
    minimum_seam = [min_energy_index]
    for y in range(height - 2, -1, -1):
        x = minimum_seam[-1]
        min_neighbor = min(total[y][max(0, x-1):min(width, x+2)])
        x = total[y].index(min_neighbor, max(0, x-1), min(width, x+2))
        minimum_seam.append(x)

    # Reverse the seam to get the bottom up path
    return minimum_seam.reverse()


# function to carve the image
def carve_column(image_array, seams_to_remove):
    for _ in range(seams_to_remove):
        # first locte the coordinates of the seam, on a map equal to the dimensions of the image
        minimum_seam = locate_seam(image_array)
        
        # overlay the seam with the original image and iterate though identically to remove the seam
        for p in range(len(image_array)):
            image_array[p].pop(minimum_seam[p])

    return image_array


# function to rotate the image array 90 degrees
def roate_image(image_array):
    return [[row[i] for row in image_array] for i in range(len(image_array[0]))]


CAS_M_processed_20_0.pgm


In [ ]:
# Part I : Vertical seam removal only
# Save your processed file to img_processed_v_h.pgm

def main():
    # create the output file
    out_file = filename.split(".")[0]+"_M_processed_"+str(verticalSeam2Remove)+"_0.pgm"
    print(out_file)

    # input the image to seam
    image_array = parse_pgm(filename)

    # preform the carve the specified number of times
    carve_column(image_array, verticalSeam2Remove)
    
    # Output the image as text
    output_file(image_array, out_file)


if __name__ == '__main__':
    main()

In [4]:
# Part II : both vertical and horizontal seams removal 
# Save your processed file to img_processed_v_h.pgm

def main():
    # create the output file
    out_file = filename.split(".")[0]+"_M_processed_"+ str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
    print(out_file)

    # input the image to seam
    image_array = parse_pgm(filename)

    # preform the carve the specified number of times
    image_array = carve_column(image_array, verticalSeam2Remove)

    # rotate the image and then seam again
    image_array = roate_image(image_array)

    # preform the carve the specified number of times, this time horizontal, because of the rotation
    image_array = carve_column(image_array, horizontalSeam2Remove)

    # rotate it back to display
    image_array = roate_image(image_array)
    
    # Output the image as text
    output_file(image_array, out_file)


if __name__ == '__main__':
    main()

CAS_2_M_processed_100_50.pgm
